<a href="https://colab.research.google.com/github/aaditya9803/ml/blob/main/Sorting_line_ml.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
import os

# Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)
folder_id = '1BZJfMkmr5uyyL1iZGPQen-B3VTYrJjD0'
# folder_id = '1bxp8LJq3826UeOR44M9ZiDFX61Dp-xSf'
folder_list = drive.ListFile({'q': "'{}' in parents and mimeType='application/vnd.google-apps.folder' and trashed=false".format(folder_id)}).GetList()


for folder in folder_list:
    folder_title = folder['title']
    print(f"Processing folder: {folder_title}")
    file_list = drive.ListFile({'q': "'{}' in parents and trashed=false".format(folder['id'])}).GetList()

    for index, file in enumerate(file_list):
        file_title = file['title']
        new_file_name = f"{folder_title}-{index}"
        print(f"Downloading {file_title} as {new_file_name}")
        file.GetContentFile(new_file_name)

Processing folder: niO
Processing folder: biO
Processing folder: riO
Processing folder: wiO


In [10]:
from google.colab import files
import numpy as np
import pandas as pd
!pip install tsfresh
from tsfresh import extract_features, select_features
from tsfresh.utilities.dataframe_functions import impute
from tsfresh.feature_selection.relevance import calculate_relevance_table
from sklearn.model_selection import train_test_split, KFold, cross_val_predict, cross_val_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, f1_score, precision_score
import xgboost as xgb

In [20]:
def read_file(filename):
    return pd.read_csv((filename), sep=',')
########################################################## INDEX ################################################

#Index to seperarate the read data into train and test dataframes
# index = index - 4
index_train = 50
index_test = 50

# Dictonaries to read the files
white_train = white_test = {}
blue_train = blue_test = {}
red_train = red_test = {}
nio_train = nio_test = {}

#Assigning df
white_train_df = white_test_df = pd.DataFrame()
blue_train_df = blue_test_df = pd.DataFrame()
red_train_df = red_test_df = pd.DataFrame()
nio_train_df = nio_test_df = pd.DataFrame()

#Reading files for training
for j in range (index_train):
  white_train[j] = read_file(f"wiO-{j}")
  blue_train[j] = read_file(f"biO-{j}")
  red_train[j] = read_file(f"riO-{j}")
  nio_train[j] = read_file(f"niO-{j}")
# print (index_train)

#Reading files for testing
for i in range (index_test):
  white_test[i] = read_file(f"wiO-{i+index_train}")
  blue_test[i] = read_file(f"biO-{i+index_train}")
  red_test[i] = read_file(f"riO-{i+index_train}")
  nio_test[i] = read_file(f"niO-{i+index_train}")
# print (i+index_train)

In [23]:
# Color df for Train Dataset
df_color_train = pd.DataFrame()
for i in range(len(white_train)):
  df_color_train = pd.concat([df_color_train, pd.Series(0)])
for i in range(len(blue_train)):
  df_color_train = pd.concat([df_color_train, pd.Series(1)])
for i in range(len(red_train)):
  df_color_train = pd.concat([df_color_train, pd.Series(2)])
for i in range(len(nio_train)):
  df_color_train = pd.concat([df_color_train, pd.Series(3)])
df_color_train = df_color_train.astype(int)
df_color_train.columns = ['color']


# Color df for Test Dataset
df_color_test = pd.DataFrame()
for i in range(len(white_test)):
  df_color_test = pd.concat([df_color_test, pd.Series(0)])
for i in range(len(blue_test)):
  df_color_test = pd.concat([df_color_test, pd.Series(1)])
for i in range(len(red_test)):
  df_color_test = pd.concat([df_color_test, pd.Series(2)])
for i in range(len(nio_test)):
  df_color_test = pd.concat([df_color_test, pd.Series(3)])
df_color_test = df_color_test.astype(int)
df_color_test.columns = ['color']

In [24]:
print(df_color_train.shape)
print(df_color_test.shape)

(200, 1)
(200, 1)


In [26]:
# Merging df of different Colors into a single df for Train Dataset

q= 0
r= index_train
s = index_train*2
t = index_train*3


for j in range(index_train):
  white_train[j] = pd.DataFrame(white_train[j])
  white_train[j]['fileno'] = q
  white_train_df = pd.concat([white_train_df, white_train[j]])

  blue_train[j] = pd.DataFrame(blue_train[j])
  blue_train[j]['fileno'] = r
  blue_train_df = pd.concat([blue_train_df, blue_train[j]])

  red_train[j] = pd.DataFrame(red_train[j])
  red_train[j]['fileno'] = s
  red_train_df = pd.concat([red_train_df, red_train[j]])

  nio_train[j] = pd.DataFrame(nio_train[j])
  nio_train[j]['fileno'] = t
  nio_train_df = pd.concat([nio_train_df, nio_train[j]])

  q = q + 1
  r = r + 1
  s = s + 1
  t = t + 1



# Merging df of different Colors into a single df for Test Dataset

q= 0
r= index_test
s = index_test*2
t = index_test*3


for j in range(index_test):
  white_test[j] = pd.DataFrame(white_test[j])
  white_test[j]['fileno'] = q
  white_test_df = pd.concat([white_test_df, white_test[j]])

  blue_test[j] = pd.DataFrame(blue_test[j])
  blue_test[j]['fileno'] = r
  blue_test_df = pd.concat([blue_test_df, blue_test[j]])

  red_test[j] = pd.DataFrame(red_test[j])
  red_test[j]['fileno'] = s
  red_test_df = pd.concat([red_test_df, red_test[j]])

  nio_test[j] = pd.DataFrame(nio_test[j])
  nio_test[j]['fileno'] = t
  nio_test_df = pd.concat([nio_test_df, nio_test[j]])

  q = q + 1
  r = r + 1
  s = s + 1
  t = t + 1


In [ ]:
# for i in range (len(white_df)):
#   white_df['fileno'] = i
# for j in range (i, i + len(blue_df)):
#   blue_df['fileno'].append(j)
# for k in range (j,j+len(red_df)):
#   red_df['fileno'].append(j)
# for l in range (k, k+len(nio_df)):
#   nio_df['fileno'].append(j)

In [ ]:
# white_df ['color'] = 0
# blue_df ['color'] = 1
# red_df ['color'] = 2
# nio_df ['color'] = 3

In [ ]:
df_train = pd.concat([white_train_df, blue_train_df, red_train_df, nio_train_df])
df_test = pd.concat([white_test_df, blue_test_df, red_test_df, nio_test_df])

# for i in range(index*4):
#   df['fileno'] = i
df_train = df_train.drop(columns=['Datum'])
x = df_train.drop(columns=['P_In(W)','P_Out(W)','Energie_In(Wh)','Energie_Out(Wh)'])
# y = df['color']
# j=0
# y_aggregated = pd.DataFrame()
# for i in range(len(df['fileno'])):
#   if df['fileno'].values[i] ==



  # if df['color'].values[i] == j:
  #   y_aggregated = pd.concat([y_aggregated, pd.Series(j)])
  # elif df['color'].values[i] != j:
  #   j = j + 1
  #   y_aggregated = pd.concat([y_aggregated, pd.Series(j)])
  # else:
  #   print('error')


In [ ]:
df.tail()

,Uhrzeit,I_In,V_In,P_In(W),I_Out(A),V_Out(V),P_Out(W),Temp(°C),Energie_In(Wh),Energie_Out(Wh),fileno
876,2024-05-02 13:05:39.262050:262,0.27,11.61,3.1347,0.25,9.00,2.2500,38.13,0.066755,0.052998,199
877,2024-05-02 13:05:39.270394:270,0.31,11.61,3.5991,0.25,9.03,2.2575,38.13,0.066764,0.053003,199
878,2024-05-02 13:05:39.282562:282,0.31,11.61,3.5991,0.31,9.00,2.7900,38.38,0.066776,0.053013,199
879,2024-05-02 13:05:39.290867:290,0.25,11.70,2.9250,0.31,9.00,2.7900,38.13,0.066782,0.053019,199
880,2024-05-02 13:05:39.303178:303,0.31,11.56,3.5836,0.25,9.00,2.2500,37.88,0.066795,0.053027,199


In [ ]:
df['fileno'].values

array([  0,   0,   0, ..., 199, 199, 199])

In [ ]:
features =  extract_features(
    x,
    column_id="fileno",
    column_sort="Uhrzeit",
    impute_function=impute
)

Feature Extraction: 100%|██████████| 1000/1000 [13:05<00:00,  1.27it/s]


In [ ]:
# feature_file = f'features-{index*4}.csv'
# features.to_csv(feature_file, sep=',')
# files.download(feature_file)

In [ ]:
features.head()

,I_In__variance_larger_than_standard_deviation,I_In__has_duplicate_max,I_In__has_duplicate_min,I_In__has_duplicate,I_In__sum_values,I_In__abs_energy,I_In__mean_abs_change,I_In__mean_change,I_In__mean_second_derivative_central,I_In__median,...,Temp(°C)__fourier_entropy__bins_5,Temp(°C)__fourier_entropy__bins_10,Temp(°C)__fourier_entropy__bins_100,Temp(°C)__permutation_entropy__dimension_3__tau_1,Temp(°C)__permutation_entropy__dimension_4__tau_1,Temp(°C)__permutation_entropy__dimension_5__tau_1,Temp(°C)__permutation_entropy__dimension_6__tau_1,Temp(°C)__permutation_entropy__dimension_7__tau_1,Temp(°C)__query_similarity_count__query_None__threshold_0.0,Temp(°C)__mean_n_absolute_max__number_of_maxima_7
0,0.0,0.0,1.0,1.0,189.99,60.2121,0.042299,0.000305,-8.051530e-06,0.29,...,1.204209,1.795131,3.805391,1.318114,2.155573,2.989938,3.756227,4.416451,0.0,37.88
1,0.0,0.0,1.0,1.0,185.58,59.6738,0.042040,0.000216,0.000000e+00,0.29,...,1.436031,2.089244,4.045644,1.410305,2.321328,3.249000,4.080992,4.797539,0.0,37.63
2,0.0,0.0,1.0,1.0,184.09,59.0125,0.043344,0.000250,-4.625929e-20,0.30,...,1.139872,1.786103,3.792321,1.462869,2.446170,3.402180,4.301580,5.059894,0.0,37.63
3,0.0,0.0,0.0,1.0,187.40,60.1416,0.042311,0.000246,-1.642036e-05,0.29,...,1.219551,1.839760,3.856714,1.485377,2.535739,3.577661,4.507263,5.212234,0.0,37.63
4,0.0,0.0,1.0,1.0,183.78,59.4834,0.041930,0.000285,1.680672e-05,0.29,...,1.211701,1.898553,3.854095,1.467557,2.476408,3.476928,4.379596,5.122465,0.0,37.63


In [ ]:
print(features.shape)
y= df_color['color']
y = y.reset_index(drop=True)
print(y.shape)

(200, 3915)
(200,)


In [ ]:

y

,color
0,0
1,0
2,0
3,0
4,0
...,...
195,3
196,3
197,3
198,3


In [ ]:
selected_features = select_features(features, y)

In [ ]:
selected_features.head()

,I_Out(A)__sum_of_reoccurring_data_points,I_In__sum_of_reoccurring_data_points,Temp(°C)__sum_of_reoccurring_data_points,"I_In__fft_aggregated__aggtype_""variance""",Temp(°C)__abs_energy,"Temp(°C)__fft_coefficient__attr_""abs""__coeff_0",Temp(°C)__sum_values,"Temp(°C)__fft_coefficient__attr_""real""__coeff_0","I_Out(A)__fft_coefficient__attr_""angle""__coeff_1",I_Out(A)__range_count__max_1__min_-1,...,"I_Out(A)__fft_coefficient__attr_""abs""__coeff_52","I_In__fft_coefficient__attr_""abs""__coeff_18","I_In__fft_coefficient__attr_""abs""__coeff_49","Temp(°C)__change_quantiles__f_agg_""mean""__isabs_True__qh_1.0__ql_0.4","I_Out(A)__fft_coefficient__attr_""abs""__coeff_75","Temp(°C)__change_quantiles__f_agg_""var""__isabs_False__qh_0.4__ql_0.2","Temp(°C)__change_quantiles__f_agg_""mean""__isabs_True__qh_0.4__ql_0.2","Temp(°C)__change_quantiles__f_agg_""var""__isabs_True__qh_0.4__ql_0.2",I_Out(A)__partial_autocorrelation__lag_4,"I_Out(A)__linear_trend__attr_""pvalue"""
0,189.20,186.58,23397.99,10023.276497,878768.6987,23397.99,23397.99,23397.99,12.281726,622.0,...,1.936640,2.818966,0.864501,0.009772,1.365618,0.026331,0.105332,0.015238,0.313336,9.732668e-12
1,183.39,182.01,22658.52,8893.805173,850026.7576,22658.52,22658.52,22658.52,13.918459,602.0,...,1.709872,2.444789,2.194649,0.115036,1.655305,0.000000,0.000000,0.000000,0.190863,2.882170e-12
2,182.81,181.48,22567.51,9239.818040,846013.1913,22567.51,22567.51,22567.51,13.999959,601.0,...,1.248486,2.373812,0.837620,0.122736,1.623350,0.000000,0.000000,0.000000,0.228127,1.075370e-11
3,183.34,183.44,22888.18,9143.044817,857410.4134,22888.18,22888.18,22888.18,14.953357,610.0,...,1.589452,2.286361,1.707862,0.120370,1.456777,0.000000,0.000000,0.000000,0.264175,1.424840e-12
4,182.58,178.62,22371.61,8209.689272,838352.7793,22371.61,22371.61,22371.61,11.591896,596.0,...,1.036741,2.623803,2.911375,0.122650,1.204775,0.000000,0.000000,0.000000,0.233434,1.172800e-09


In [ ]:
x_train,x_test,y_train,y_test = train_test_split(selected_features,y, test_size=0.5, random_state=42, stratify=y)

In [ ]:
model = RandomForestClassifier()
model.fit(x_train,y_train)
y_pred = model.predict(x_test)
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy}")
print(classification_report(y_test, y_pred))
print(confusion_matrix(y_test, y_pred))

Accuracy: 0.97
              precision    recall  f1-score   support

           0       0.96      1.00      0.98        25
           1       0.96      0.96      0.96        25
           2       1.00      0.96      0.98        25
           3       0.96      0.96      0.96        25

    accuracy                           0.97       100
   macro avg       0.97      0.97      0.97       100
weighted avg       0.97      0.97      0.97       100

[[25  0  0  0]
 [ 0 24  0  1]
 [ 1  0 24  0]
 [ 0  1  0 24]]


In [ ]:
def xgb_(X, y, num_folds=5):

    kf = KFold(n_splits=num_folds, shuffle=True, random_state=24)

    f1_scores = []
    accuracies = []
    precisions = []

    for fold_idx, (train_idx, test_idx) in enumerate(kf.split(X)):
        print(f"Fold {fold_idx + 1}:")

        X_train, X_test = X.iloc[train_idx], X.iloc[test_idx]
        y_train, y_test = y.iloc[train_idx], y.iloc[test_idx]

        # Initialize and train the XGBoost model with regularization
        xgb1 = xgb.XGBClassifier(
            objective="multi:softmax",
            random_state=42,
            alpha=0.1,
            reg_lambda=0.1,
            max_depth=4,
            min_child_weight=10,
            subsample=0.8,
            colsample_bytree=0.8
        )

        # Fit the model
        xgb1.fit(X_train.to_numpy(), y_train.to_numpy().ravel())

        # Make predictions
        y_pred = xgb1.predict(X_test.to_numpy())

        # Ensure y_test is 1D for comparison
        y_test = y_test.to_numpy().ravel().astype(int)
        y_pred = y_pred.astype(int)

        # Calculate and store F1-score for this fold
        f1 = f1_score(y_test, y_pred, average='weighted')
        f1_scores.append(f1)

        # Calculate and store accuracy and precision for this fold
        accuracy = accuracy_score(y_test, y_pred)
        accuracies.append(accuracy)
        precision = precision_score(y_test, y_pred, average='weighted')
        precisions.append(precision)

        print(classification_report(y_test, y_pred))
        print(f"Fold {fold_idx + 1} F1-score: {f1:.4f}")
        print(f"Fold {fold_idx + 1} Accuracy: {accuracy:.4f}")
        print(f"Fold {fold_idx + 1} Precision: {precision:.4f}")

        # Calculate and print confusion matrix
        cm = confusion_matrix(y_test, y_pred)
        print("Confusion Matrix:")
        print(cm)

        print("-" * 30)

    # Calculate and print average metrics across all folds
    avg_f1 = np.mean(f1_scores)
    avg_accuracy = np.mean(accuracies)
    avg_precision = np.mean(precisions)

    print(f"Average F1-score across {num_folds} folds: {avg_f1:.4f}")
    print(f"Average Accuracy across {num_folds} folds: {avg_accuracy:.4f}")
    print(f"Average Precision across {num_folds} folds: {avg_precision:.4f}")

    # Final model: Train on the entire dataset
    print("Training final model on the entire dataset...")
    X_train_final, X_test_final, y_train_final, y_test_final = train_test_split(X, y, test_size=0.2, random_state=42)

    # Final model with regularization
    final_model = xgb.XGBClassifier(
        objective="multi:softmax",
        random_state=42,
        alpha=0.1,
        reg_lambda=0.1,
        max_depth=5,
        min_child_weight=10,
        subsample=0.8,
        colsample_bytree=0.8
    )

    # Train on the entire dataset
    final_model.fit(X_train_final.to_numpy(), y_train_final.to_numpy().ravel())

    # Evaluate the final model on the test set
    y_pred_final = final_model.predict(X_test_final.to_numpy())

    # Calculate accuracy, precision, confusion matrix for final model
    final_accuracy = accuracy_score(y_test_final, y_pred_final)
    final_precision = precision_score(y_test_final, y_pred_final, average='weighted')
    final_cm = confusion_matrix(y_test_final, y_pred_final)

    print(f"Final Model Accuracy: {final_accuracy:.4f}")
    print(f"Final Model Precision: {final_precision:.4f}")
    print("Final Model Confusion Matrix:")
    print(final_cm)

    # Save the final model
    # with open('XGB_final_model.pkl', 'wb') as f:
    #     pickle.dump(final_model, f)
    # files.download('XGB_final_model.pkl')



In [ ]:
xgb_ (selected_features, y)

Fold 1:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00         8
           1       1.00      1.00      1.00        10
           2       1.00      1.00      1.00         9
           3       1.00      1.00      1.00        13

    accuracy                           1.00        40
   macro avg       1.00      1.00      1.00        40
weighted avg       1.00      1.00      1.00        40

Fold 1 F1-score: 1.0000
Fold 1 Accuracy: 1.0000
Fold 1 Precision: 1.0000
Confusion Matrix:
[[ 8  0  0  0]
 [ 0 10  0  0]
 [ 0  0  9  0]
 [ 0  0  0 13]]
------------------------------
Fold 2:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00         9
           1       1.00      1.00      1.00        10
           2       1.00      1.00      1.00        10
           3       1.00      1.00      1.00        11

    accuracy                           1.00        40
   macro avg       1.00      1.00      1